In [1]:
### Preface
import javabridge
import bioformats as bf
javabridge.start_vm(class_path=bf.JARS)
import numpy as np
import pandas as pd
from os.path import isfile
import matplotlib.pyplot as plt

from sys import path as syspath
syspath.append("./functions/")
from physio_def_1 import getApparentFreq, importFrames, getTimes

from plotFirst_1 import plotImage

In [2]:
from collections import OrderedDict

In [3]:
pathToFile = '/Volumes/physio/team/slakrupnik/project/experiments/MB/Cytokines/cytokines-1.lif'
# pathToFile = "/Volumes/physio/team/slakrupnik/project/experiments/NIKON/20190806/512X512_image_slice3_16la_2ca.nd2"
# pathToFile = '/Users/srdjan/Documents/Physio_Ca/data/Experiment1/Experiment1.lif'

md = bf.get_omexml_metadata(pathToFile)
xml = bf.OMEXML(md)
Nimages = xml.get_image_count()
rdr = bf.ImageReader(pathToFile, perform_init=True)
extension = "."+pathToFile.split(".")[-1]

In [4]:
OrderedDict([(i,xml.image(i).Name) for i in range(Nimages)])

OrderedDict([(0, 'Image002'), (1, 'Series003'), (2, 'Image004')])

In [6]:
import os.path as ospath

In [9]:
ospath.split(pathToFile)

('/Volumes/physio/team/slakrupnik/project/experiments/MB/Cytokines',
 'cytokines-1.lif')

In [5]:
for iSeries in range(Nimages):
    im = xml.image(iSeries)
    Name = im.Name
    
    if Nimages==1:
        npzFile   = pathToFile.replace(extension,f".npz")
        imageFile = pathToFile.replace(extension,f".png")
        dimFile   = pathToFile.replace(extension,f".txt")
    else:
        npzFile   = pathToFile.replace(extension,f"_{iSeries}_{Name}.npz")
        imageFile = pathToFile.replace(extension,f"_{iSeries}_{Name}.png")
        dimFile   = pathToFile.replace(extension,f"_{iSeries}_{Name}.txt")
    
    if isfile(npzFile):
        continue
    
    dimensions = dict(zip("TXY",(getattr(im.Pixels, "Size"+dim) for dim in "TXY")))
    if dimensions["T"]>1:
        dimensions['freq'] = getApparentFreq(iSeries,xml)

    howManyFirstFrames = min(10,dimensions["T"])
    firstFrames = importFrames(rdr,idx=iSeries, which=(howManyFirstFrames,))

    extraDims = len(firstFrames.shape)>3
    if extraDims:
#         print("There are more dimensions here than expected.\nI'll remedy this for now, but be shure to keep this in mind.")
        retainDims = np.where(firstFrames.std(axis=tuple(range(len(firstFrames.shape)-1)))>0)[0]
        assert len(retainDims)==1
        retainDims = retainDims[0]
        firstFrames = firstFrames.T[retainDims].T
    else:
        retainDims = None

    meanFirstFrames = firstFrames.mean(axis=0)
#     stdFirstFrames = firstFrames.std(axis=0)

    pxSize = im.Pixels.get_PhysicalSizeX()
    pxUnit = im.Pixels.get_PhysicalSizeXUnit()
    text = "\n".join([" %s:%i"%(c,dimensions[c])  for c in "XYT"])
    if dimensions["T"]>1:
        text += "\n f:%.1f Hz"%dimensions['freq']
    dimensions["pxSize"] = pxSize
    dimensions["pxUnit"] = pxUnit


    pd.DataFrame([dimensions]).to_csv(dimFile)
    plotImage(meanFirstFrames.T,pxSize=pxSize,pxUnit=pxUnit,
              savePath=imageFile,
              addInfo=text)
    plt.close()
    
        
    time = getTimes(xml_=xml,idx_=iSeries)

    mintype = pd.to_numeric(firstFrames.flatten(),downcast="unsigned").dtype
    allimage = importFrames(
        rdr,
        idx=iSeries,
        dtype=mintype,
        which=(dimensions['T'],None,None,1)[:3+int(extraDims)]
    #     which=(None,None,None)+tuple([retainDims][:int(extraDims)])
    )
    np.savez_compressed(npzFile,time=time,data=allimage)

./functions/physio_def_1.py:95: UserWarning: Could not give all required time points. I advise you double check the output
  warn(f"Could not give all required time points. I advise you double check the output")


KeyboardInterrupt: 